### Get Input File & Setup

In [1]:
table=[
    [1,1,1,1,1],
    [1,1,1,0,1],
    [1,0,1,1,1],
    [0,1,1,1,1],
    [0,1,1,0,1],
    [1,0,1,0,0],
    [0,0,0,1,1],
    [0,0,0,0,0]
]

B = {
    'A','B','C','D','E'
}

FactorHash = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4
}

NegFactorSet = {}

### Perform Alg on File

In [2]:
#STEP 0

# FactorTable is a hashtable/dictionary of all factors, factorA -> index | key -> value
# For everything in W, we will have to test this
# Over every factor, we are collecting the set of factors that are possible effects, this set is W
# B: set of all factors, main_table: C
def step0(main_table, B, FactorTable, NegFactorSet):
    print("mainTable:")
    print(main_table)
    W = []
    for factor in B:
        print("Looking at " + factor)
        #(2)
        if rowDuplicity(main_table, FactorTable[factor]):
            print(factor + " failed row duplicity")
            continue
        #(3)
        elif isNeg(FactorTable[factor], NegFactorSet):
            print(factor + " failed isNeg")
            continue
        W.append(factor)
    return W

'''
    Check if the same row but negative factor already exists, if not, insert into set, else return 1
    int[i][j] table, i= coincidence, j = factor index
'''
def rowDuplicity(table, factor):
    rowSet = []
    #loop through every row in table
    for coincidence in table:
        modif_coincidence = coincidence.copy()
        modif_coincidence[factor] = 1-modif_coincidence[factor]
        if modif_coincidence in rowSet:
            return 1
        rowSet.append(coincidence)
        
def isNeg(factor, NegFactorSet):
    if factor in NegFactorSet:
        return 1
    return 0

In [3]:
potential_effects = step0(table, B, FactorHash, NegFactorSet)

mainTable:
[[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 0]]
Looking at D
D failed row duplicity
Looking at A
A failed row duplicity
Looking at B
B failed row duplicity
Looking at C
Looking at E


### Main Loop

In [45]:
#-- HELPER FUNCTIONS - SELF SET FUNCTIONS --##

def setIn(fauxSet, coincidence):
    #loop through fauxSet
#     print("---------")
#     print(coincidence)
#     print(fauxSet)
#     print("---------")
    for elem in fauxSet:
        for index, fac in enumerate(elem):
            #compare each index in fauxElement to coincidence
            if(coincidence[index] != fac):
                break
            if index == len(elem)-1:
                return True
    return False
        

In [46]:

#-------------------------------------------------------------------------------
#STEP 1
#FOR LOOP
# [[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1], 
#  [0, 0, 0, 0, 0]]
# C
# []
# [[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 0]]
# E
# []
#--------------------------------
#STEP 2

from numpy import *

'''
table: int[][] : double array of 1/0
factor: int : index of Z_i 
coincidence: X_k Z_i
modif_coincidence: X_k !Z_i
returns set of rows where coincidence X_k is sufficient for Z_i
'''
def step2(main_table, factor):
    rowSet = []
    rowNegSet = []
    for coincidence in table:
        
        modif_coincidence = coincidence.copy() 
        modif_coincidence[FactorHash[factor]] = 1 - modif_coincidence[FactorHash[factor]]
        
        if coincidence[FactorHash[factor]]==1:   #if Z_i is true add to set of sufficient rows unless modif_coincidence is in the table
            if not setIn(rowNegSet, modif_coincidence):
                rowSet.append(coincidence)
                
        elif coincidence[FactorHash[factor]]==0: #if Z_i is false remove modif_coincidence from sufficient rows
            if setIn(rowSet, modif_coincidence):
                rowSet.remove(coincidence)
            rowNegSet.append(coincidence)
    return rowSet
#--------------------------------
#STEP 3

from itertools import permutations

class sufficient_objects:
    def __init__(self):
        #one coincidence
        self.coincidence = []
        #array of permutation set
        self.permutations = []

def step3(rowSet, main_table, effect):
    coincidences_list = []
    
#     coinc_array = [coindicends]
#     arary_set = [{tmp}]
    
#     coinc[5] = [100101]
#     array_set[5] = [{11000}{00110}{00001}]
#     for set in array_set[5]:
#         getval(coinc[5],set)
        
    for coincidence in rowSet:
        coincidence_obj = sufficient_objects()
        suff_set = {}
        prm_list = list(permutations(range(0,len(coincidence))))
        # for every permutation of coincidence
        for prm in prm_list:
            #remove Z_g
            tmp_prm = prm.copy()
            for factor in tmp_prm:
                #remove
                prm.remove(factor)
                #is remaining prm suff?
                if not check_sufficient(coincidence, prm, table, effect):
                    prm.append(factor)
            suff_set.add(prm)
        coincidence_obj.coincidence = coincidence
        coincidence_obj.permutations = suff_set
        coincidences_list.append(coincidence_obj)
                    
    return coincidences_list
                    
# returns 1 if sufficient
def check_sufficient(coincidence, prm, table, effect):
    extravar = false
    for row in table:
        var = true
        for col in row:
            if col.index in prm:
                if col != coincidence[col.index]:
                    var = false
                    break
        if var is true:
            extravar = true
            #in order for the original coincidence to be sufficient, its effect had to have been 1
            if row[effect] == 0:
                return 0
    return extravar



In [5]:
#--------------------------------
#STEP 4
#END OF FOR LOOP -- WE ARE IGNORING THIS STEP FOR EFFICIENCY

#--------------------------------
#STEP 5

def step5(main_table, sufficient_objs, effect):
    for obj in sufficient_objs:
        for prm in obj.permutations:
            is_necessary = true
            for row in main_table:
                if row[effect] == 1:
                    for index in prm:
                        if obj.coincidence[index] != row[index]:
                            is_necessary = false
                            break
            if !is_necessary:
                obj.permuations.remove(prm)
    return sufficient_objs

#--------------------------------
#STEP 6

from itertools import permutations

def step6(rowSet, main_table, effect)
    for coincidence in rowSet:
        prm_list = list(permutations(range(0,coincidence.length)))
        # for every permutation of coincidence
        for prm in prm_list:
            #remove Z_g
            for factor in prm:
                #remove
                removeprm()
                #is remaining prm suff?
                if !check_necessary(table, coincidence, mod_prm, effect):
                    addbackprm()
                    
def check_necessary(main_table, coincidence, prm, effect):
    for row in main_table:
        if row[effect] == 1:
            for index in prm:
                if coincidence[index] != row[index]:
                    return false
    return true

SyntaxError: invalid syntax (<ipython-input-5-894eea004f88>, line 18)

In [48]:
for effect in potential_effects:
    rowSet = step2(table,effect)
#     sufficientSet = step3(rowSet,table,effect)
    print(effect)
    print(rowSet)

C
[[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0]]
E
[[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [0, 0, 0, 1, 1]]


### Framing